## 27.3 Static analysis

The next section will show three problems that can't be solved by any algorithm.
The reason is because those problems are about functions,
not data (numbers, strings, graphs, etc.).
So let's first look more closely at that kind of problem in this section.
Since algorithms for Turing machines are unwieldy to write,
let's return to Python.

### 27.3.1 Functions on functions

Consider the following problem.
It takes as input a function, not a value of some data type.

**Function**: get info\
**Inputs**: *f*, a Python function\
**Preconditions**: true\
**Output**: *text*, a string\
**Postconditions**: *text* is the header and docstring of *f*

This problem is solved by the built-in function `help`, introduced in
[Section&nbsp;2.6.1](../02_Sequence/02_6_py_functions.ipynb#2.6.1-Documentation).

In [1]:
help(len)

Help on built-in function len in module builtins:

len(obj, /)
    Return the number of items in a container.



Another example of functions being inputs was given in
[Section&nbsp;14.1.4](../14_Sorting/14_1_sort_prep.ipynb#14.1.4-Sorting-in-Python):
Python's `sort` method and `sorted` function can take as argument
a function that for each item computes the key to be used for sorting.
The example was to sort cards by suit or by value.

In [2]:
%run -i ../m269_sorting        # defines key functions for sorting

sorted(["2S", "AS", "2D", "AD"], key=suit)  # diamonds, then spades

['2D', 'AD', '2S', 'AS']

In [3]:
sorted(["2S", "AS", "2D", "AD"], key=value)  # aces, then twos

['AS', 'AD', '2S', '2D']

Function `help` can also be applied to our functions, not just those built-in.

In [4]:
help(suit)  # replace suit with value to see what value does

Help on function suit in module __main__:

suit(card: str) -> str
    Return the second character of the card.

    Preconditions: card has two characters;
    the first is 'A', '2' to '9', 'T', 'J', 'Q' or 'K'
    the second is 'C', 'D', 'H' or 'S'



A further example of a 'function on functions' is `test`, used throughout this book:
it takes as inputs a test table and the function to be tested.

As a final example, consider retrieving the function's source code,
including the header and docstring.

**Function**: get code\
**Inputs**: *f*, a Python function\
**Preconditions**: the source code of *f* is available\
**Output**: *text*, a string\
**Postconditions**: *text* is the source code of *f*

This problem is solved by function `getsource` in module `inspect`.
It doesn't work for built-in functions like `len` because their source code
isn't available, but it works for code we wrote.

In [5]:
from inspect import getsource

print(getsource(suit))  # replace suit with value if you wish

def suit(card: str) -> str:
    """Return the second character of the card.

    Preconditions: card has two characters;
    the first is 'A', '2' to '9', 'T', 'J', 'Q' or 'K'
    the second is 'C', 'D', 'H' or 'S'
    """
    return card[1]



You can also look at the source code of imported functions, like `test`,
but it's likely they will contain Python constructs you haven't learned.

Python provides useful functions that operate on functions, like `help` and `getsource`, but even
more useful is the ability to write such functions ourselves.

### 27.3.2 Writing functions on functions

When we write an assignment like `x = 5`, the Python interpreter allocates
some memory and stores there an object representing 5. The object not only
contains the value 5 but also information that it's of type `int`. Finally,
the interpreter creates pointer `x` to associate name `x` to
the memory address where the created object is stored.

The interpreter must store the object's type so that it knows
what operations are valid. When we write `x + 1`, the interpreter follows the
`x` pointer to find the associated object, retrieves the type, which is `int`,
and confirms the addition operation is defined.
Whereas if we write `len(x)`, the interpreter detects that because `x` is
an integer, it has no operation `len` and reports a type error.

Similarly, when we write `def suit(card: str) ...`, the interpreter
allocates memory, stores the function there together with information that
this object is of type `Callable`
([Section&nbsp;14.3.2](../14_Sorting/14_3_insertion_sort.ipynb#14.3.2-Iterative-version)) and
creates pointer `suit`, pointing at the stored function.
When we write `suit('2S')`, the interpreter follows the `suit` pointer,
confirms the associated object is of type `Callable`, i.e. a function, and then
passes the string to the function and executes it.

Therefore, there's nothing magic about `help`, `getsource` and `sorted`.
They simply take as input a pointer to a `Callable` object and then
extract the docstring or source code of the stored function or, in the case of
`sorted`, execute the function on each item to obtain its sorting keys.
Our sorting functions in Chapter&nbsp;14 do exactly that:
they take a key function as input and apply it to the items to sort them.
Here again is our [insertion sort](../14_Sorting/14_3_insertion_sort.ipynb#14.3.2-Iterative-version) code,
without docstring and comments.

In [6]:
from typing import Callable


def insertion_sort(items: list, key: Callable) -> None:  # noqa: D103
    for first_unsorted in range(1, len(items)):
        to_sort = items[first_unsorted]
        the_key = key(to_sort)
        index = first_unsorted
        while index > 0 and key(items[index - 1]) > the_key:
            items[index] = items[index - 1]
            index = index - 1
        items[index] = to_sort

Notice the calls `key(to_sort)` and `key(items[index - 1])` to the function
`key` given as an argument.

### 27.3.3 Navel gazing

If a function takes *any* function as input, then it can be applied to itself.

In [7]:
help(help)

Help on _Helper in module _sitebuiltins object:

class _Helper(builtins.object)
 |  Define the builtin 'help'.
 |
 |  This is a wrapper around pydoc.help that provides a helpful message
 |  when 'help' is typed at the Python interactive prompt.
 |
 |  Calling help() at the Python prompt starts an interactive help session.
 |  Calling help(thing) prints help for the python object 'thing'.
 |
 |  Methods defined here:
 |
 |  __call__(self, *args, **kwds)
 |      Call self as a function.
 |
 |  __repr__(self)
 |      Return repr(self).
 |
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |
 |  __dict__
 |      dictionary for instance variables
 |
 |  __weakref__
 |      list of weak references to the object



Never mind what the output says. The point of this example is that
`help` follows *any* pointer to a function in order to retrieve its header and docstring.
Therefore, it can follow the pointer to itself.

Functions that 'look into' functions without running them do **static analysis**
of the code. Here's an example of another static analysis function:
it very crudely attempts to check if a function iterates.

In [8]:
def has_loop(function: Callable) -> bool:
    """Check if the code of `function` includes 'while' or 'for' and 'in'."""
    text = getsource(function)
    return " while " in text or (" for " in text and " in " in text)

Insertion sort does a while-loop, so it has a loop.

In [9]:
has_loop(insertion_sort)

True

Our key functions don't have a loop: they don't iterate over
the 2-character string representing the card.

In [10]:
has_loop(suit)  # replace suit with value if you wish

False

Since `has_loop` takes any function with available source code,
it can be called on itself.

In [11]:
has_loop(has_loop)

True

We get the wrong answer because the function just looks for the characters
`while`, `for` and `in` in the source code, without distinguishing their
occurrence as a keyword, in a string or in a comment.

Function `has_loop` isn't meant to illustrate robust static analysis,
but rather that we *can* write functions that analyse others and themselves.
As the next section will show, there are limits to the power of static analysis.

#### Exercise 27.3.1 (optional)

Write a function that checks if the given input function has a docstring.
Test the function on the other functions in this notebook and on itself.

⟵ [Previous section](27_2_thesis.ipynb) | [Up](27-introduction.ipynb) | [Next section](27_4_undecidability.ipynb) ⟶